<h2> Spectroscopic data reduction : spectra module

<b><i> This is an older, mostly obsolete notebook. See notebooks for KOSMOS, ARCES, and DIS instead. Here there is a small TSPEC section saved until a full TSPEC notebook is developed

In [1]:
%matplotlib qt
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import os

Set up a reducer and tv object

In [59]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos')
t=tv.TV()

INSTRUMENT: KOSMOS   config: 
  will use format:  /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/*{:04d}.fits*
         gain:  [0.6]    rn: [5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


Make flat field, and remove spectral shape (spec=True)

In [3]:
flat=red.mkflat([18,19,20,21,22],spec=True,display=t)

  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/flat.0018.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/flat.0019.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/flat.0020.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/flat.0021.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/flat.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....
  See final image, use - key for S/N image.
   To continue, hit space in display window (p for debug) 
    see image: 18 divided by master
   To continue, hit space in display window (p for debug) 
    see image: 19 divided by 

/home/local/Anaconda3-2020.02/lib/python3.7/site-packages/numpy/core/fromnumeric.py:746: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


Read and display a star spectral image

In [60]:
star=red.reduce(11)
t.tv(star,max=10000)

  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/BD28d4211.0011.fits
  subtracting overscan vector 
  subtracting overscan vector 


We want to extract the spectrum, i.e. from the 2D image to a 1D spectrum. Start by defining a trace that is just constant along rows, and extract the spectrum with a 25 pixel window.

In [61]:
# here we set a trace to be at a constant position on the detector
t.clear()
def model(x) : return(np.zeros(len(x))+940)
trace=spectra.Trace(rows=[550,1450],lags=range(-300,300),model=[model])
starec=trace.extract(red.imtranspose(star),plot=t,rad=25)

  See extraction window(s). Hit space bar to continue....



Inspection of the trace and window shows that the stellar spectrum is not perfectly horizontal. So we will trace out its location (row as a function of column), and extract around that trace.

In [62]:
# here we solve for the trace using a star images
trace.trace(red.imtranspose(star),[940],sc0=2048,plot=t)
starec=trace.extract(red.imtranspose(star),plot=t,rad=25)

/home/holtz/git/pyvista/python/pyvista/spectra.py:710: RuntimeWarning: invalid value encountered in true_divide
  res = model(cols)-ypos
/home/holtz/git/pyvista/python/pyvista/spectra.py:710: RuntimeWarning: invalid value encountered in greater
  res = model(cols)-ypos
/home/holtz/git/pyvista/python/pyvista/spectra.py:715: RuntimeWarning: invalid value encountered in true_divide
  #model=copy.copy(oldmodel[irow])
/home/holtz/git/pyvista/python/pyvista/spectra.py:715: RuntimeWarning: invalid value encountered in greater
  #model=copy.copy(oldmodel[irow])



  See trace. Hit space bar to continue....
  See extraction window(s). Hit space bar to continue....



Now let's turn to wavelength calibration, i.e. getting a function that gives the wavelength as a function of pixel. We'll solve for this using arc frames, here taken with each lamp separately, so sum the three exposures

In [63]:
#Frame 15 is He, 16 is Ne, and 17 is Ar
arcs=red.sum([15,16,17])
t.clear()
t.tv(arcs)

  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/He.0015.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/Ne.0016.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/Ar.0017.fits
  subtracting overscan vector 
  subtracting overscan vector 


In [64]:
arcec=trace.extract(red.imtranspose(arcs),plot=t,rad=20)

  See extraction window(s). Hit space bar to continue....



In [9]:
#Rhis cell translates old pickle file to fits

#traces=pickle.load(open(files(pyvista.data).joinpath('KOSMOS/KOSMOS_traces.pkl'),'rb'))
#traces[0][0].degree=traces[0][0].order
#traces[0][0].spectrum=traces[0][0].spectrum.data
#traces[0][0].type='Polynomial1D'
#coeffs=[]
#for m in traces[0][0].model :
#    coeffs.append([m.c0.value,m.c1.value,m.c2.value])
#traces[0][0].coeffs = np.array(coeffs)

#tab=traces[0][0].write(files(pyvista.data).joinpath('KOSMOS/KOSMOS_traces.fits'))


Wavelength calibration first stars with identifying lines. This is much easier if one can work from a previous solution. Here we start by reading a previous solution into a pyvista WaveCal object.

In [66]:
trace=spectra.Trace(file=files(pyvista.data).joinpath('KOSMOS/KOSMOS_traces.fits'))
#wav=pickle.load(open(files(pyvista.data).joinpath('KOSMOS/KOSMOS_blue_waves.pkl'),'rb'))
wav=spectra.WaveCal(file=files(pyvista.data).joinpath('KOSMOS/KOSMOS_red_waves.fits'))

doing wavelength fit
  rms:    0.076 Angstroms


In [67]:
#wav[0][0].write(files(pyvista.data).joinpath('KOSMOS/KOSMOS_blue_waves.fits'))

#tmp=spectra.WaveCal(file=files(pyvista.data).joinpath('KOSMOS/KOSMOS_blue_waves.fits'))

#vars(tmp)
#wav[0][0]=tmp

With a previous solution loaded, the identify routine will cross correlate the input spectrum with the previous solution, then attempt to identify the lines from the previous solution at the shifted pixel position from the previous solution. Finally, it does a fit.

In [68]:
fig,ax=plots.multi(1,2,figsize=(12,6))
wav.identify(arcec,plot=fig,rad=10)

  cross correlating with reference spectrum using lags:  range(-300, 300)
  Derived pixel shift from input wcal:  [0.14880354]
  See identified lines. Hit space bar in plot window to continue....
doing wavelength fit
  rms:    0.125 Angstroms
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    0.125 Anstroms
  input from plot window...
1 5886.656749000611
  rms:    0.073 Anstroms
  input from plot window...



The previous wavelength file we read in set the model to be a 3rd order Chebyshev polynomial. Is this order needed?

In [70]:
wav.fit(degree=3)

doing wavelength fit
  rms:    0.136 Angstroms
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    0.073 Anstroms
  input from plot window...


We can get the rough dispersion from the first order coefficient. However, the astropy routines map the pixels of the fit lines to the range [-1,1], so we need to account for that to get A/pixel from the first order coefficient

In [15]:
# approximate dispersion
for key,value in vars(wav.model).items() :
    print(key,value)
domain=wav.model.domain[1]-wav.model.domain[0]
window=wav.model.window[1]-wav.model.window[0]
print('rough dispersion: ',wav.model.c1/(domain/window))

domain [350.1212149849779, 3088.243128125736]
window [-1, 1]
_degree 3
_order 4
_param_names ('c0', 'c1', 'c2', 'c3')
_inputs ('x',)
_outputs ('y',)
_name None
c0 Parameter('c0', value=7243.233233332454)
c1 Parameter('c1', value=-1413.790403804372)
c2 Parameter('c2', value=17.022500481622853)
c3 Parameter('c3', value=5.9966694899777675)
_mconstraints {'eqcons': [], 'ineqcons': []}
_model_set_axis False
_param_metrics defaultdict(<class 'dict'>, {'c0': {'slice': slice(0, 1, None), 'shape': (), 'size': 1}, 'c1': {'slice': slice(1, 2, None), 'shape': (), 'size': 1}, 'c2': {'slice': slice(2, 3, None), 'shape': (), 'size': 1}, 'c3': {'slice': slice(3, 4, None), 'shape': (), 'size': 1}})
_n_models 1
_parameters [ 7.24323323e+03 -1.41379040e+03  1.70225005e+01  5.99666949e+00]
_input_units_strict {'x': False}
_input_units_allow_dimensionless {'x': False}
rough dispersion:  -1.0326716257733655


The identify() method fits Gaussians to the lines to get their position, and gets FWHM in the process. Plot the FWHM of the lines as a function of wavelength

In [16]:
wav.fwhm
t.plotax2.cla()
t.plotax2.plot(wav.waves,np.abs(wav.fwhm),'ro')

If se didn't have a previous solution, we can start by identifying a couple of lines and using them to calculate an approximate dispersion and one (wave,pixel) pair. Here, we'll cheat and use two lines from the previous solution.
<img src=henear.png>

In [17]:
w0=wav.waves[0]
w1=wav.waves[-5]
pix0=int(wav.pix[0])
pix1=int(wav.pix[-5])
print(w0,pix0,w1,pix1)
disp=(w1-w0)/(pix1-pix0)

5852.4878 3088 8424.647 582


Now, we start a new WaveCal object,try to identify the lines with our initial (wave,pixel) and dispersion estimate, and see how the fit comes out.

In [72]:
w=spectra.WaveCal(degree=3)
w.identify(arcec,wref=[w0,pix0],disp=disp,thresh=100,
           plot=fig,pixplot=True,file='henearkr.dat',rad=10)

  See identified lines. Hit space bar in plot window to continue....
doing wavelength fit
  rms:    7.907 Angstroms
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    7.907 Anstroms


/home/local/Anaconda3-2020.02/lib/python3.7/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


  input from plot window...
  rms:    4.984 Anstroms
  input from plot window...
38 8807.017776012544
  rms:    3.489 Anstroms
  input from plot window...
34 8623.393856988288
  rms:    0.083 Anstroms
  input from plot window...



Not very well! However, one can see that there's a good sequence of lines on the red side of the chip, so we can remove all of the others, get a higher order solution from these good ones, and iterate.

In [74]:
w.identify(arcec,file='henearkr.dat')

  cross correlating with reference spectrum using lags:  range(-300, 300)
  Derived pixel shift from input wcal:  [-6.80410039e-09]
doing wavelength fitin row:  0
  rms:    1.546 Angstroms
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    1.546 Anstroms
  input from plot window...
5 5876.664874554946
  rms:    1.563 Anstroms
  input from plot window...
9 5979.265106961306
  rms:    1.542 Anstroms
  input from plot window...
4 5872.3179475709585
  rms:    1.353 Anstroms
  input from plot window...
4 5872.669034694058
  rms:    1.353 Anstroms
  input from plot window...
6 5885.4928486446815
  rms:    1.215 Anstroms
  input from plot window...
3 5868.091394387632
  rms:    0.172 Anstroms
  input from plot window...
0 5009.553786336475
  rms:    0.132 Anstroms
  input from plot window...
7 5943.993561600419
  rms:  

OK, now use the wavelength solution to get wavelength as a function of pixel

In [22]:
wav.wave(image=arcec.data.shape).shape
import matplotlib.pyplot as plt
waves=wav.wave(image=arcec.data.shape)[0]

# get inverse relation, i.e. pixels as f(wavelength)
pix=np.arange(4096)
from scipy.interpolate import CubicSpline
wav2pix=CubicSpline(np.flip(waves),np.flip(pix))
plt.figure()
plt.plot(waves)

Make plots of He, Ne, and Ar spectra, labelling with wavelength

In [23]:
#reduce
he=red.reduce(15)
ne=red.reduce(16)
ar=red.reduce(17)
#extract
heec=trace.extract(red.imtranspose(he),rad=20)
neec=trace.extract(red.imtranspose(ne),rad=20)
arec=trace.extract(red.imtranspose(ar),rad=20)
#plot
plt.figure()
plt.clf()
plt.plot(pix,heec.data[0],color='b',label='He')
plt.plot(pix,neec.data[0],color='g',label='Ne')
plt.plot(pix,arec.data[0],color='r',label='Ar')

  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/He.0015.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/Ne.0016.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/holtz/raw/apo/oct21/Q4NM01/UT211030/kosmos/Ar.0017.fits
  subtracting overscan vector 
  subtracting overscan vector 
  extracting aperture 0
  extracting aperture 0
  extracting aperture 0


Label the lines

In [24]:
#line labels
for line in wav.waves :
    plt.text(wav2pix(line),0,'{:.2f}'.format(line),rotation=90,va='top',ha='center')
    
#plot limits and axes, and legend
plt.ylim(-1.5e6,5e6)
frame=plt.gca()
frame.axes.get_xaxis().set_visible(True)
frame.axes.get_yaxis().set_visible(False)
plt.legend()
plt.xlabel('Pixel')
plt.tight_layout()


Wavelength calibration for star spectrum

In [26]:
#plt.figure()
plt.clf()
plt.plot(waves,starec.data[0])
print(1/(5.5e-5*np.log(10)))
wnew=10**np.arange(3.5,4.0,5.5e-6)
plt.plot(wnew,wav.scomb(starec,wnew).data)

7896.263307331851


/home/holtz/git/pyvista/python/pyvista/spectra.py:558: RuntimeWarning: invalid value encountered in true_divide
  out = out / sig
/home/holtz/git/pyvista/python/pyvista/spectra.py:559: RuntimeWarning: divide by zero encountered in true_divide
  sig = np.sqrt(1./sig)


<h3> longslit extraction and wavelength calibration

For extended objects, and perhaps for sky subtraction, we might want to work along the slit. The wavelength solution varies along the slit (line curvature), usually with more than just an offset.

Start by working along the slit to identify lines

In [27]:
from pyvista import image
arcec=trace.extract2d(red.imtranspose(arcs)) 
image.smooth(arcec,[5,1])
w.identify(spectrum=arcec, rad=3,display=t, plot=fig,
              nskip=5,lags=np.arange(-50,50))

  cross correlating with reference spectrum using lags:  [-50 -49 -48 -47 -46 -45 -44 -43 -42 -41 -40 -39 -38 -37 -36 -35 -34 -33
 -32 -31 -30 -29 -28 -27 -26 -25 -24 -23 -22 -21 -20 -19 -18 -17 -16 -15
 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2   3
   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21
  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  44  45  46  47  48  49]
  Derived pixel shift from input wcal for row: 899 4
  See identified lines. Hit space bar in plot window to continue....



In [28]:
w.fit()

doing wavelength fit
  rms:    0.116
rejecting 57 points from 6119 total: 
  rms:    0.109
rejecting 60 points from 6119 total: 
  rms:    0.109
rejecting 59 points from 6119 total: 
  rms:    0.109
rejecting 59 points from 6119 total: 
  See 2D wavecal fit. Hit space bar in plot window to continue....


OK, use the solution to make a wavelength map

In [29]:
waves2d=w.wave(image=arcec.data.shape)
t.clear()
t.tv(waves2d)

Subtract out the central wavelength solution to see how the solution varies with row

In [30]:
dw=waves2d*0.
for row in np.arange(waves2d.shape[0]) :
    dw[row] = waves2d[row]-waves2d[450]
t.tv(dw)   

<h2> ARCES : echelle spectroscopy

Set up the reducer

In [31]:
red=imred.Reducer('ARCES',dir='UT211031/echelle')

INSTRUMENT: ARCES   config: 
  will use format:  UT211031/echelle/*{:04d}.fits*
         gain:  [3.8]    rn: [7.]
         scale:  None   
  Biastype : 0
  Bias box: 
    SC    NC    SR    NR
  2075    51    20  2009 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  2048 
  Norm box: 
    SC    NC    SR    NR
  1000    51  1000    51 


Create biases and flats. Note that for flats, we have to do scattered light removal, which can be done on reduction of individual images, but since it is slow, we will do it on the combined flat.

In [32]:
bias=red.mkbias([19,20,21,22,23,24])
flat_red=red.mkflat([5,6,7,8,9,10,11])
flat_blue=red.mkflat([12,13,14,15,16,17,18])

# do scatter after combination to save time
red.scatter(flat_red,scat=red.scat)
red.scatter(flat_blue,scat=red.scat)

  Reading file: UT211031/echelle/bias.0019.fits
  subtracting overscan:  1288.7897402863584
  Reading file: UT211031/echelle/bias.0020.fits
  subtracting overscan:  1288.5753423320548
  Reading file: UT211031/echelle/bias.0021.fits
  subtracting overscan:  1288.5232629637221
  Reading file: UT211031/echelle/bias.0022.fits
  subtracting overscan:  1288.5549244087879
  Reading file: UT211031/echelle/bias.0023.fits
  subtracting overscan:  1288.9714031954245
  Reading file: UT211031/echelle/bias.0024.fits
  subtracting overscan:  1289.0347651255624
  combining data with median....
  calculating uncertainty....
  Reading file: UT211031/echelle/flat_red.0005.fits
  subtracting overscan:  1287.6691164270587
  Reading file: UT211031/echelle/flat_red.0006.fits
  subtracting overscan:  1287.7610166017625
  Reading file: UT211031/echelle/flat_red.0007.fits
  subtracting overscan:  1287.5283284045327
  Reading file: UT211031/echelle/flat_red.0008.fits
  subtracting overscan:  1287.6828585092574
 

Combine the red and blue flats.

In [52]:
flat=flat_red.multiply(flat_red.header['MEANNORM'])
flat.add(flat_blue.multiply(flat_blue.header['MEANNORM']))
flat.divide(flat_red.header['MEANNORM']+flat_blue.header['MEANNORM'])
t.clear()
t.tv(flat_blue)
t.tv(flat_red)
t.tv(flat)
t.tv(flat,sn=True)

For tracing and wavelength calibration, we will use previously determined traces and line identification as a first guess.

In [ ]:
#this cell translates old pkl file to fits
#traces=pickle.load(open(files(pyvista.data).joinpath('ARCES/ARCES_traces.pkl'),'rb'))

#traces[0][0].degree=traces[0][0].order
#traces[0][0].spectrum=traces[0][0].spectrum.data
#traces[0][0].type='Polynomial1D'
#coeffs=[]
#for m in traces[0][0].model :
#    coeffs.append([m.c0.value,m.c1.value,m.c2.value])
#traces[0][0].coeffs = np.array(coeffs)
#
#tab=traces[0][0].write(files(pyvista.data).joinpath('ARCES/ARCES_traces.fits'))


In [34]:
from importlib_resources import files
traces=spectra.Trace(file=files(pyvista.data).joinpath('ARCES/ARCES_traces.fits'))
#wav=pickle.load(open(files(pyvista.data).joinpath('ARCES/ARCES_wave.pkl'),'rb'))
wav=spectra.WaveCal(file=files(pyvista.data).joinpath('ARCES/ARCES_wave.fits'))

doing wavelength fit
  rms:    0.019
rejecting 90 points from 3653 total: 
  rms:    0.019
rejecting 90 points from 3653 total: 


In [35]:
#wav[0][0].write(files(pyvista.data).joinpath('ARCES/ARCES_wave.fits'))
#tmp=spectra.WaveCal(file='junk.fits')
#vars(tmp)
#wav[0][0]=tmp

Sum up the arc lamp exposures, get the shift for the existing traces, and extract.

In [36]:
arc=red.sum([1,2,3,4],crbox=[5,1])
traces.find(arc,plot=t)
arcec=traces.extract(arc,plot=t)

  Reading file: UT211031/echelle/ThAr.0001.fits
  subtracting overscan:  1287.781893245103
  zapping CRs with filter [5,1]...
  Reading file: UT211031/echelle/ThAr.0002.fits
  subtracting overscan:  1287.783181565309
  zapping CRs with filter [5,1]...
  Reading file: UT211031/echelle/ThAr.0003.fits
  subtracting overscan:  1287.6101074576172
  zapping CRs with filter [5,1]...
  Reading file: UT211031/echelle/ThAr.0004.fits
  subtracting overscan:  1287.7658966025435
  zapping CRs with filter [5,1]...
  See spectra and cross-correlation. Hit any key in display window to continue....
  See extraction window(s). Hit space bar to continue....



Do line identification based on previously identified lines, and wavelength fit.

In [53]:
fig,ax=plots.multi(1,2,figsize=(12,6))
wav.identify(spectrum=arcec,rad=3,display=t,plot=fig)

  cross correlating with reference spectrum using lags:  range(-300, 300)
  Derived pixel shift from input wcal:  [-5.25183168e-08]
  See spectrum and template spectrum (top), cross correlation(bottom). hit any key in display window to continue
  See identified lines. Hit space bar in plot window to continue....
doing wavelength fit
  rms:    0.008
rejecting 35 points from 2004 total: 
  rms:    0.007
rejecting 43 points from 2004 total: 
  rms:    0.006
rejecting 47 points from 2004 total: 
  rms:    0.006
rejecting 55 points from 2004 total: 
  rms:    0.006
rejecting 55 points from 2004 total: 
  See 2D wavecal fit. Hit space bar in plot window to continue....



Now reduce an image

In [40]:
im=red.reduce(28,scat=red.scat,display=t)

  Reading file: UT211031/echelle/HIP109807.0028.fits
  subtracting overscan:  1291.718306834929
  See bias box and cross section. 
   To continue, hit space in display window (p for debug) 
  estimating scattered light ...
    fitting surface ...
    rejecting points ...
    refitting surface ...
  See image with scattered light points
   To continue, hit space in display window (p for debug) 
  See scattered light image
   To continue, hit space in display window (p for debug) 


Get shift of traces, and extract

In [41]:
traces.find(im,plot=t)
imec=traces.extract(im,plot=t)

  See spectra and cross-correlation. Hit any key in display window to continue....
  See extraction window(s). Hit space bar to continue....



Get the wavelengths for all pixels from the wavelength solution and plot extracted spectra.

In [46]:
w=wav.wave(image=imec.data.shape)
print(w.shape)
print(type(imec))
import matplotlib.pyplot as plt
plt.figure()
for row in range(w.shape[0]) :
    plt.plot(w[row],imec.data[row])

(107, 2128)
<class 'astropy.nddata.ccddata.CCDData'>


Now do 1D flat fielding. 2D flat fielding would be better, but poses challenges with ARCES because the traces move, and there's no way to illuminate all pixels.

In [49]:
traces.find(flat)
flat1d=traces.extract(flat)
t.tv(flat1d)

  extracting aperture 106


Resample onto logarithmic wavelength grid and combine orders

In [48]:
wnew=10**np.arange(3.5,4.0,5.5e-6)
comb=wav.scomb(imec.divide(flat1d),wnew,average=True,usemask=True)
plt.clf()
plt.plot(wnew,comb.data)

/home/holtz/git/pyvista/python/pyvista/spectra.py:558: RuntimeWarning: invalid value encountered in true_divide
  out = out / sig
/home/holtz/git/pyvista/python/pyvista/spectra.py:559: RuntimeWarning: divide by zero encountered in true_divide
  sig = np.sqrt(1./sig)


In [ ]:
from pyvista import reduce
fig=plt.figure()
reduce.all('oct21.yml',groups='ARCES',display=None,plot=fig)

<h2> TSPEC : multiple orders

In [ ]:
tspec=imred.Reducer(inst='TSPEC',dir='UT191026/TSPEC',nfowler=8)
a=tspec.reduce(21)
dark=tspec.reduce(22)
t.clear()
t.tv(a)
a=a.subtract(dark)
t.tv(a)

In [ ]:
rows=[[135,235],[295,395],[435,535],[560,660],[735,830]]

apers=[155,316,454,591,761]
apers.reverse()
t.clear()
t.tv(a)
traces=spectra.Trace(order=3,rows=rows,lags=range(-75,75))
traces.trace(a,apers,sc0=350,plot=t)
traces=[traces]


In [ ]:
traces[0].pix0

In [ ]:
import scipy
acr=tspec.reduce(21,crbox=[11,1])
t.clear()
t.tv(acr)
trace=traces[0]
trace.pix0=30
order=7
for aper,row in zip(apers,rows) :
    out=trace.extract(acr,rad=20,plot=t)
    #out.data=out.data - scipy.signal.medfilt(out.data,kernel_size=[1,201])
    #wcal=spectra.WaveCal(type='chebyshev',orders=[order],degree=3)
    #w=np.atleast_2d(wav[order-3,0,:][::-1])*1.e4
    #bd=np.where(~np.isfinite(w))
    #w[bd[0],bd[1]]=9000.


